<a href="https://colab.research.google.com/github/CaseyAnnHorvath/desihigh/blob/main/two_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/desihigh/')

In [3]:
import time
import astropy
import itertools
import matplotlib

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [4]:
from IPython.display import clear_output
from time import sleep

In [5]:
%matplotlib inline

plt.style.use('dark_background')

In [6]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [7]:
npt = 2
pos, vel  = init_dof(npt=npt)
mass=np.zeros_like(pos)


In [8]:
i = 0
pos[i][0]=0
pos[i][1]=0
pos[i][2]=0
pos[0]

i = 1
pos[i][0]=1.5e12
pos[i][1]=0
pos[i][2]=0



# vel of particle 1
i=0

vel[i][0]=0
vel[i][1]=11000
vel[i][2]=0

i=1

vel[i][1]=0
vel[i][1]=-5500
vel[i][2]=0



mass[0]=1.5e30
mass[1]=6e30

In [35]:
def com(pos,mass):  #defining CoM function, 
  result=np.dot(mass*pos)/np.sum(mass)
  return result

In [36]:
#Rcom=com(pos,mass)

TypeError: ignored

In [37]:
print, Rcom

(<function print>, array([4.e+11, 0.e+00, 0.e+00]))

In [12]:
pos_com=pos-Rcom
print, pos_com

(<function print>, array([[-4.0e+11,  0.0e+00,  0.0e+00],
        [ 1.1e+12,  0.0e+00,  0.0e+00]]))

In [48]:
#defining original values for the 2 bodies

m1=1.5e30
m2=6e30

v1=1.1e4
v2=-5.5e3

r1=0
r2=1.5e12

#distance between m1 and m2
r=r2-r1


In [49]:
def com_vel(vel,mass):
  result=np.sum(mass*vel,axis=0)/np.sum(mass)
  return result

#Simple 2body version

def findCoM(m1, r1, v1, m2, r2, v2):
  return (m1*r1 + m2*r2)/(m1+m2), (m1*v1 + m2*v2)/(m1+m2)

In [50]:
#dont need anymore
#print(findCoM(m1, r1, v1, m2, r2, v2)) #1.5e30, 0, 11000, 6e30, 1.5e12, -5500
#print, com_vel(vel,mass)

In [51]:
com_pos, com_vel = findCoM(m1, r1, v1, m2, r2, v2)
print(com_pos, com_vel)

1200000000000.0 -2200.0


In [52]:
#setting com reference frome to 0 vel

v1=v1-com_vel
v2=v2-com_vel

print(v1,v2)

13200.0 -3300.0


In [53]:
#Gravitational Constant
G=6.67e-11

In [54]:
#energy calculation


def energy(m1,v1,m2,v2,r):
  return .5*m1*(v1**2)+.5*m2*(v2**2)-((G*m1*m2)/r)
  

In [56]:
E=energy(m1,v1,m2,v2,r)
print(E)

-2.3684999999999995e+38


In [57]:
#angular momentum calculation
def ang_momentum(m1,m2,v1,v2,r1,r2):
  return m1*v1*r1+m2*v2*r2

In [58]:
L=ang_momentum(m1,m2,v1,v2,r1,r2)
print(L)

-2.9699999999999997e+46


In [59]:
#calc reduced mass
def reduced_mass(m1,m2):
  return (m1*m2)/(m1+m2)
  

In [60]:
u=reduced_mass(m1,m2)
print(u)

1.1999999999999998e+30


In [61]:
#calc total mass
def tot_mass(m1,m2):
  return m1+m2

In [62]:
M=tot_mass(m1,m2)
print(M)

7.5e+30


In [63]:
#calc eccentricity
def eccentricity(L,E,u,M):
  return (1+(2*(L**2)*E)/((u**3)*(G**2)*(M**2)))**(1/2)

In [64]:
e=eccentricity(L,E,u,M)
print(e)

0.183658170914542


In [65]:
#calc semi major axis
def sem_maj(M,u,E):
  return (G*M*u)/(2*E)
  

In [66]:
a=sem_maj(M,u,E)
print(a)

-1267257758074.731


In [67]:
#calculating distance for perhelion (maybe dont need)
def perihelion_rad(a,e):
  return a*(1-e)

r_peri=perihelion_rad(a,e)
print(r_peri)

-1034515516149.4628


In [68]:
#calculating distance for aphelion (maybe dont need)
def aphelion_rad(a,e):
  return a*(1+e)

r_ap=aphelion_rad(a,e)
print(r_ap)

-1499999999999.9993


In [69]:
#calc location of m1 and m2 (maybe dont need)


In [104]:
#calc theta-theta knot 

def d_theta(L,u,M,r,e,G):
  arg=((L**2)/(G*M*(u**2)*r*e))-(1.0/e)
  
  return np.arccos(np.around(arg,4))


In [105]:
d_theta=d_theta(L,u,M,r,e,G)
print(d_theta)

3.141592653589793


In [106]:
def roh(L,u,M,r,e,G,d_theta,theta):
  return ((L**2)/(G*M*(u**2)*(1.0+e*np.cos(theta+d_theta))))

In [107]:
roh= roh(L,u,M,r,e,G,d_theta,0)
print(roh)

1499999999999.9988
